In [1]:
import requests
import bs4
import os
import re
import unicodedata
import pandas as pd
import nltk
import acquire as a

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns


import sys
sys.path.insert(1, '/Users/matthewdalton/')
from env import github_token, github_username

from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords


In [2]:
url = ('https://github.com/microsoft?q=&type=&language=&sort=stargazers')
response = requests.get(url)
html = response.text
soup = bs4.BeautifulSoup(html)

In [3]:
REPOS = []

In [4]:
soup.select('h3')[0].text

'\n\n            vscode\n'

In [5]:
for n in range(0,30):
    REPOS.append(soup.select('h3')[n].text)
    REPOS = list(set(REPOS))

In [6]:
len(REPOS)

30

In [7]:
i = 2
while i < 35:
    url = 'https://github.com/microsoft?language=&page=' + str(i) + '&sort=stargazers&type='
    response = requests.get(url)
    html = response.text
    soup = bs4.BeautifulSoup(html)
    
    for n in range(0,30):
        REPOS.append(soup.select('h3')[n].text)
        REPOS = list(set(REPOS))
        
    i += 1

In [8]:
REPOS[:10]

['\n\n            monaco-typescript\n',
 '\n\n            WSL-DistroLauncher\n',
 '\n\n            lain\n',
 '\n\n            FastTrack\n',
 '\n\n            Oscar\n',
 '\n\n            service-fabric-services-and-actors-dotnet\n',
 '\n\n            vision-ai-developer-kit\n',
 '\n\n            CNTK\n',
 '\n\n            angle\n',
 '\n\n            LightGBM\n']

In [9]:
regexp = r'[\n]+\s+'
subject = REPOS[0]


re.sub(regexp, '', subject)

'monaco-typescript\n'

In [10]:
repos = pd.Series(REPOS)

In [11]:
repos = repos.str.replace(regexp, "", regex=True)

In [12]:
repos = repos.str.replace('Archived\n', '').str.replace('\n', '')

In [13]:
REPOS = list('microsoft/' + repos)

In [14]:
REPOS

['microsoft/monaco-typescript',
 'microsoft/WSL-DistroLauncher',
 'microsoft/lain',
 'microsoft/FastTrack',
 'microsoft/Oscar',
 'microsoft/service-fabric-services-and-actors-dotnet',
 'microsoft/vision-ai-developer-kit',
 'microsoft/CNTK',
 'microsoft/angle',
 'microsoft/LightGBM',
 'microsoft/AutonomousDrivingCookbook',
 'microsoft/MLOpsPython',
 'microsoft/knack',
 'microsoft/vscode-sublime-keybindings',
 'microsoft/roosterjs',
 'microsoft/appcenter-sdk-apple',
 'microsoft/MSBuildLocator',
 'microsoft/ailab',
 'microsoft/BingMapsRESTToolkit',
 'microsoft/recommenders',
 'microsoft/cppwin32',
 'microsoft/Azure_Kinect_ROS_Driver',
 'microsoft/WindowsTemplateStudio',
 'microsoft/checkedc-llvm',
 'microsoft/L4',
 'microsoft/llvm-mctoll',
 'microsoft/cookiecutter-spacy-fastapi',
 'microsoft/frameworkcontroller',
 'microsoft/mail2bug',
 'microsoft/service-fabric',
 'microsoft/playwright-sharp',
 'microsoft/vscode-docs',
 'microsoft/2D-TAN',
 'microsoft/fluentui-system-icons',
 'microsoft/

In [15]:
import os
import json
from typing import Dict, List, Optional, Union, cast
import requests

import sys
sys.path.insert(1, '/Users/matthewdalton/')

from env import github_token, github_username


headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}

if headers["Authorization"] == "token " or headers["User-Agent"] == "":
    raise Exception(
        "You need to follow the instructions marked TODO in this script before trying to use it"
    )


def github_api_request(url: str) -> Union[List, Dict]:
    response = requests.get(url, headers=headers)
    response_data = response.json()
    if response.status_code != 200:
        raise Exception(
            f"Error response from github api! status code: {response.status_code}, "
            f"response: {json.dumps(response_data)}"
        )
    return response_data


def get_repo_language(repo: str) -> str:
    url = f"https://api.github.com/repos/{repo}"
    repo_info = github_api_request(url)
    if type(repo_info) is dict:
        repo_info = cast(Dict, repo_info)
        if "language" not in repo_info:
            raise Exception(
                "'language' key not round in response\n{}".format(json.dumps(repo_info))
            )
        return repo_info["language"]
    raise Exception(
        f"Expecting a dictionary response from {url}, instead got {json.dumps(repo_info)}"
    )


def get_repo_contents(repo: str) -> List[Dict[str, str]]:
    url = f"https://api.github.com/repos/{repo}/contents/"
    contents = github_api_request(url)
    if type(contents) is list:
        contents = cast(List, contents)
        return contents
    raise Exception(
        f"Expecting a list response from {url}, instead got {json.dumps(contents)}"
    )


def get_readme_download_url(files: List[Dict[str, str]]) -> str:
    """
    Takes in a response from the github api that lists the files in a repo and
    returns the url that can be used to download the repo's README file.
    """
    for file in files:
        if file["name"].lower().startswith("readme"):
            return file["download_url"]
    return ""


def process_repo(repo: str) -> Dict[str, str]:
    """
    Takes a repo name like "gocodeup/codeup-setup-script" and returns a
    dictionary with the language of the repo and the readme contents.
    """
    contents = get_repo_contents(repo)
    readme_download_url = get_readme_download_url(contents)
    if readme_download_url == "":
        readme_contents = ""
    else:
        readme_contents = requests.get(readme_download_url).text
    return {
        "repo": repo,
        "language": get_repo_language(repo),
        "readme_contents": readme_contents,
    }


def scrape_github_data() -> List[Dict[str, str]]:
    """
    Loop through all of the repos and process them. Returns the processed data.
    """
    return [process_repo(repo) for repo in REPOS]


if __name__ == "__main__":
    data = scrape_github_data()
    json.dump(data, open("data.json", "w"), indent=1)

In [16]:
scrape_github_data()

ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [17]:
f = open('data.json')

In [18]:
# returns JSON object as 
# a dictionary
data = json.load(f)

In [19]:
df = pd.DataFrame(data)

In [20]:
df

,repo,language,readme_contents
0,microsoft/monaco-typescript,JavaScript,# Monaco TypeScript\n\nSimple TypeScript and J...
1,microsoft/WSL-DistroLauncher,C++,# WSL Distro Launcher Reference Implementation...
2,microsoft/lain,Rust,# lain\n\nThis crate provides functionality on...
3,microsoft/FastTrack,PowerShell,# Microsoft FastTrack Open Source\nWelcome to ...
4,microsoft/Oscar,Python,# Oscar: Object-Semantics Aligned Pre-training...
...,...,...,...
1015,microsoft/vscode-npm-scripts,TypeScript,# Node npm\n\n![vscode version](https://vsmark...
1016,microsoft/Product-Recommendations,C#,"<img src=""images/logo.png"" align=""right"" heig..."
1017,microsoft/aspnet-api-versioning,C#,[![Build status](https://microsoft.visualstudi...
1018,microsoft/language-server-protocol-inspector,Vue,"<p>\n <h2 align=""center"">Language Server Prot..."


In [21]:
df.language.value_counts()

C#                  230
TypeScript          163
C++                 115
Python              112
JavaScript           71
PowerShell           37
Jupyter Notebook     32
C                    32
Java                 24
Go                   18
HTML                 18
CSS                  10
Shell                 9
Rust                  8
Dockerfile            7
Objective-C           6
PHP                   5
Ruby                  4
F#                    3
Kotlin                2
Swift                 2
R                     2
Scala                 2
Vue                   2
Perl                  2
LLVM                  2
CoffeeScript          2
Objective-C++         2
TSQL                  2
Rich Text Format      2
HCL                   2
Assembly              1
Julia                 1
Q#                    1
OCaml                 1
Liquid                1
SCSS                  1
Batchfile             1
M4                    1
Handlebars            1
AL                    1
CMake           